# Loop 2 Analysis: External Dataset Ensemble

Create a comprehensive ensemble from ALL external data sources:
1. bucket-of-chump dataset
2. telegram-shared solutions (71.97, 72.49)
3. santa25-public dataset
4. santa-2025-try3 dataset
5. SmartManoj GitHub
6. All 78 snapshots

In [1]:
import numpy as np
import pandas as pd
import os
import json
import glob
from shapely.geometry import Polygon
from shapely import STRtree

os.chdir('/home/code')

# Tree vertices
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

def get_tree_polygon(x, y, deg):
    """Get tree polygon at position (x, y) with rotation deg"""
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    vertices = []
    for j in range(len(TX)):
        vx = TX[j] * c - TY[j] * s + x
        vy = TX[j] * s + TY[j] * c + y
        vertices.append((vx, vy))
    return Polygon(vertices)

def score_group(xs, ys, degs):
    """Calculate score for a single N-tree configuration"""
    n = len(xs)
    all_x, all_y = [], []
    for i in range(n):
        rad = np.radians(degs[i])
        c, s = np.cos(rad), np.sin(rad)
        for j in range(len(TX)):
            x = TX[j] * c - TY[j] * s + xs[i]
            y = TX[j] * s + TY[j] * c + ys[i]
            all_x.append(x)
            all_y.append(y)
    side = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
    return side * side / n

def check_overlaps(xs, ys, degs):
    """Check if any trees overlap using Shapely"""
    polygons = [get_tree_polygon(xs[i], ys[i], degs[i]) for i in range(len(xs))]
    tree = STRtree(polygons)
    for i, poly in enumerate(polygons):
        candidates = tree.query(poly)
        for j in candidates:
            if i < j:
                if polygons[i].intersection(polygons[j]).area > 0:
                    return True
    return False

def parse_submission(df):
    """Parse submission dataframe"""
    df = df.copy()
    df['x_val'] = df['x'].astype(str).str.replace('s', '').astype(float)
    df['y_val'] = df['y'].astype(str).str.replace('s', '').astype(float)
    df['deg_val'] = df['deg'].astype(str).str.replace('s', '').astype(float)
    df['n'] = df['id'].str.split('_').str[0].astype(int)
    return df

def calculate_per_n_scores(df):
    """Calculate score for each N value"""
    per_n_scores = {}
    for n in range(1, 201):
        group = df[df['n'] == n]
        if len(group) == n:
            xs = group['x_val'].values
            ys = group['y_val'].values
            degs = group['deg_val'].values
            per_n_scores[n] = score_group(xs, ys, degs)
    return per_n_scores

print("Functions defined.")

Functions defined.


In [2]:
# Collect ALL CSV files from external datasets
print("Collecting all CSV files from external datasets...")

external_csvs = []

# 1. bucket-of-chump
external_csvs.extend(glob.glob('/home/code/external_data/bucket-of-chump/*.csv'))

# 2. telegram-shared
external_csvs.extend(glob.glob('/home/code/external_data/telegram-shared/*.csv'))

# 3. santa25-public
external_csvs.extend(glob.glob('/home/code/external_data/santa25-public/*.csv'))

# 4. santa-2025-try3
external_csvs.extend(glob.glob('/home/code/external_data/santa-2025-try3/*.csv'))

# 5. SmartManoj GitHub (already downloaded)
if os.path.exists('/home/code/experiments/002_external_ensemble/smartmanoj_submission.csv'):
    external_csvs.append('/home/code/experiments/002_external_ensemble/smartmanoj_submission.csv')

print(f"Found {len(external_csvs)} external CSV files:")
for f in external_csvs:
    print(f"  {f}")

Found 22 external CSV files:
  /home/code/external_data/bucket-of-chump/submission.csv
  /home/code/external_data/telegram-shared/72.49.csv
  /home/code/external_data/telegram-shared/71.97.csv
  /home/code/external_data/santa25-public/submission_JKoT4.csv
  /home/code/external_data/santa25-public/New_Tree_144_196.csv
  /home/code/external_data/santa25-public/submission_JKoT3.csv
  /home/code/external_data/santa25-public/santa2025_ver2_v61.csv
  /home/code/external_data/santa25-public/submission_JKoT2.csv
  /home/code/external_data/santa25-public/santa2025_ver2_v67.csv
  /home/code/external_data/santa25-public/santa2025_ver2_v76.csv
  /home/code/external_data/santa25-public/submission_70_936673758122.csv
  /home/code/external_data/santa25-public/santa2025_ver2_v65.csv
  /home/code/external_data/santa25-public/submission_70_926149550346.csv
  /home/code/external_data/santa25-public/santa2025_ver2_v66.csv
  /home/code/external_data/santa25-public/santa2025_ver2_v63.csv
  /home/code/extern

In [3]:
# Load all submissions
submissions = {}

# Load external CSVs
print("\nLoading external submissions...")
for fp in external_csvs:
    try:
        df = pd.read_csv(fp)
        if {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            df = parse_submission(df)
            name = os.path.basename(fp).replace('.csv', '')
            submissions[name] = df
            print(f"  Loaded: {name} ({len(df)} rows)")
    except Exception as e:
        print(f"  Failed: {fp} - {e}")

# Load current best ensemble
print("\nLoading current ensemble...")
df_current = pd.read_csv('/home/submission/submission.csv')
df_current = parse_submission(df_current)
submissions['current_ensemble'] = df_current

# Load all snapshots
print("\nLoading snapshot submissions...")
snapshot_dir = '/home/nonroot/snapshots/santa-2025'
for snap_id in os.listdir(snapshot_dir):
    sub_path = os.path.join(snapshot_dir, snap_id, 'submission', 'submission.csv')
    if os.path.exists(sub_path):
        try:
            df = pd.read_csv(sub_path)
            df = parse_submission(df)
            submissions[f'snapshot_{snap_id}'] = df
        except:
            pass

print(f"\nTotal submissions loaded: {len(submissions)}")


Loading external submissions...
  Loaded: submission (20100 rows)
  Loaded: 72.49 (20100 rows)


  Loaded: 71.97 (20100 rows)
  Loaded: submission_JKoT4 (20100 rows)
  Loaded: New_Tree_144_196 (20100 rows)
  Loaded: submission_JKoT3 (20100 rows)
  Loaded: santa2025_ver2_v61 (20100 rows)


  Loaded: submission_JKoT2 (20100 rows)
  Loaded: santa2025_ver2_v67 (20100 rows)
  Loaded: santa2025_ver2_v76 (20100 rows)
  Loaded: submission_70_936673758122 (20100 rows)


  Loaded: santa2025_ver2_v65 (20100 rows)
  Loaded: submission_70_926149550346 (20100 rows)
  Loaded: santa2025_ver2_v66 (20100 rows)
  Loaded: santa2025_ver2_v63 (20100 rows)


  Loaded: santa2025_ver2_v69 (20100 rows)
  Loaded: submission_JKoT1 (20100 rows)
  Loaded: submission_opt1 (20100 rows)
  Loaded: santa2025_ver2_v68 (20100 rows)


  Loaded: submission (20100 rows)
  Loaded: submission_sa (20100 rows)
  Loaded: smartmanoj_submission (20100 rows)

Loading current ensemble...

Loading snapshot submissions...



Total submissions loaded: 98


In [4]:
# Calculate per-N scores for all submissions
print("Calculating per-N scores for all submissions...")
all_per_n_scores = {}

for name, df in submissions.items():
    try:
        scores = calculate_per_n_scores(df)
        if len(scores) == 200:  # Valid submission
            all_per_n_scores[name] = scores
            total = sum(scores.values())
            print(f"  {name}: total={total:.6f}")
    except Exception as e:
        print(f"  {name}: ERROR - {e}")

print(f"\nValid submissions: {len(all_per_n_scores)}")

Calculating per-N scores for all submissions...


  submission: total=72.935294
  72.49: total=72.495739


  71.97: total=71.972027


  submission_JKoT4: total=72.489504


  New_Tree_144_196: total=72.927920


...

  snapshot_21090949260: total=84.711359


  snapshot_21165874980: total=70.630478


  snapshot_21165870845: total=70.676102


  snapshot_21117525284: total=70.676104


  snapshot_21198927060: total=70.624381

Valid submissions: 98


In [ ]:
# Find best VALID (no overlap) solution for each N
print("\nFinding best VALID solution for each N (checking overlaps)...")

best_valid_per_n = {}
best_valid_source = {}

for n in range(1, 201):
    best_score = float('inf')
    best_src = None
    
    # Check all submissions for this N
    for name, df in submissions.items():
        if name not in all_per_n_scores:
            continue
        
        group = df[df['n'] == n]
        if len(group) != n:
            continue
            
        xs = group['x_val'].values
        ys = group['y_val'].values
        degs = group['deg_val'].values
        
        # Check for overlaps
        if check_overlaps(xs, ys, degs):
            continue
        
        score = all_per_n_scores[name][n]
        if score < best_score:
            best_score = score
            best_src = name
    
    if best_src is not None:
        best_valid_per_n[n] = best_score
        best_valid_source[n] = best_src
    else:
        print(f"WARNING: No valid solution for N={n}!")
        # Use current ensemble as fallback
        best_valid_source[n] = 'current_ensemble'
        best_valid_per_n[n] = all_per_n_scores['current_ensemble'][n]

# Calculate total score
total_valid = sum(best_valid_per_n.values())
print(f"\nBest VALID ensemble total score: {total_valid:.6f}")
print(f"Target: 68.894234")
print(f"Gap to target: {total_valid - 68.894234:.6f}")

In [ ]:
# Show improvements from external datasets
print("\nImprovements from external datasets:")
current_scores = all_per_n_scores.get('current_ensemble', {})
current_total = sum(current_scores.values())

improvements = []
for n in range(1, 201):
    if n in current_scores and n in best_valid_per_n:
        diff = current_scores[n] - best_valid_per_n[n]
        if diff > 1e-10:
            improvements.append((n, diff, best_valid_source[n]))

improvements.sort(key=lambda x: x[1], reverse=True)
print(f"Total N values improved: {len(improvements)}")
print(f"Total improvement: {current_total - total_valid:.6f}")

print("\nTop 30 improvements:")
for n, diff, src in improvements[:30]:
    print(f"  N={n}: improved by {diff:.6f} from {src}")

In [ ]:
# Show source distribution
print("\nSource distribution for best solutions:")
source_counts = {}
for n, src in best_valid_source.items():
    source_counts[src] = source_counts.get(src, 0) + 1

for src, count in sorted(source_counts.items(), key=lambda x: -x[1]):
    print(f"  {src}: {count} N values")

In [ ]:
# Create the valid ensemble submission
print("Creating valid ensemble submission...")

new_valid_rows = []
for n in range(1, 201):
    best_src = best_valid_source[n]
    df_best = submissions[best_src]
    group = df_best[df_best['n'] == n]
    for _, row in group.iterrows():
        new_valid_rows.append({
            'id': row['id'],
            'x': row['x'],
            'y': row['y'],
            'deg': row['deg']
        })

df_valid = pd.DataFrame(new_valid_rows)
print(f"Valid ensemble shape: {df_valid.shape}")

In [ ]:
# Final verification - check no overlaps in valid ensemble
print("Final verification - checking no overlaps...")
df_valid_parsed = parse_submission(df_valid)

has_overlap = False
overlap_count = 0
for n in range(1, 201):
    group = df_valid_parsed[df_valid_parsed['n'] == n]
    xs = group['x_val'].values
    ys = group['y_val'].values
    degs = group['deg_val'].values
    
    if check_overlaps(xs, ys, degs):
        print(f"  WARNING: Overlap detected in N={n}!")
        has_overlap = True
        overlap_count += 1

if not has_overlap:
    print("  ✓ No overlaps detected - submission is valid!")
else:
    print(f"  ✗ {overlap_count} N values have overlaps!")

In [ ]:
# Calculate and save final score
final_valid_scores = calculate_per_n_scores(df_valid_parsed)
final_valid_total = sum(final_valid_scores.values())
print(f"\nFinal VALID ensemble score: {final_valid_total:.6f}")
print(f"Previous best: {current_total:.6f}")
print(f"Improvement: {current_total - final_valid_total:.6f}")
print(f"Target: 68.894234")
print(f"Gap to target: {final_valid_total - 68.894234:.6f}")

# Save submission
df_valid.to_csv('/home/submission/submission.csv', index=False)
print("\nSaved to /home/submission/submission.csv")